# 实体对齐

In [9]:
include("../src/CCKS-mKGQA.jl")
include("../src/extractdata.jl")
include("../src/translatedata.jl")
include("../src/ettalign.jl")
mkpath.("EA_data/" .* (["", "ILLs_1", "ILLs_2", "ILLs_official", "ILLs_wiki"]))

5-element Vector{String}:
 "EA_data"
 "EA_data/ILLs_1"
 "EA_data/ILLs_2"
 "EA_data/ILLs_official"
 "EA_data/ILLs_wiki"

### 初始化

#### 模型对齐-1

In [22]:
# ILLs => 对齐字典
path = "ILLs_1"
newILLs = readtuples("EA_data/$path/ETT_Pairs.txt";size=2)
filter!(i->i[1] != "Gregoire_Kayibanda", newILLs);

#### 模型对齐-2

In [26]:
# ILLs => 对齐字典
path = "ILLs_2"
newILLs = readtuples("EA_data/$path/ILLs_predict-7.6.txt"; size=2);

#### 模型对齐-3

In [57]:
# ILLs => 对齐字典
path = "ILLs_3"
newILLs = readtuples("EA_data/$path/ILLs_predict_zh_en.txt"; size=2)
wordset = Set(words)
filter!(i->i[1] ∈ wordset, newILLs)
# filter(i->i[1] ∉ wordset, newILLs)

48131-element Vector{Tuple{String, String}}:
 ("Corrales_de_Duero", "科拉莱斯德杜埃罗")
 ("Fairview,_Montana", "费尔维尤_(蒙大拿州)")
 ("Wells_(UK_Parliament_constituency)", "韋爾斯_(英國國會選區)")
 ("Golden_Beach,_Florida", "金沙滩_(佛罗里达州)")
 ("Tokyo_Juliet_(TV_series)", "東方茱麗葉")
 ("Ibagué", "伊瓦格")
 ("Springville,_Utah", "史普林维尔_(犹他州)")
 ("Pyin_Oo_Lwin_District", "彬乌伦县")
 ("Scotland_national_under-21_football_team", "蘇格蘭21歲以下足球代表隊")
 ("Middle_East", "中东")
 ("Huntingdon_and_Peterborough", "亨廷登-彼得伯勒郡")
 ("Brasília_de_Minas", "巴西利亚-迪米纳斯")
 ("Dhofar_Governorate", "佐法爾省")
 ⋮
 ("Henagar,_Alabama", "黑纳格_(阿拉巴马州)")
 ("Men_in_Black_II", "黑色會")
 ("The_Snowmen", "黑道家族")
 ("Black_Hawk_(Amtrak_train)", "黑鷹計劃")
 ("Nouvion-sur-Meuse", "默兹河畔巴祖瓦勒")
 ("Delta_Amacuro", "黛兒塔")
 ("Zapadni_Mojstir", "鼠标垫")
 ("Led_Zeppelin_II", "齐柏林飞船_(乐团)")
 ("August_Hlond", "齐格蒙特·奥古斯特")
 ("Zimány", "齐马尼")
 ("Łagówek", "龔偉怡")
 ("Kamei_Korenori", "龜井茲矩")

#### 原始 ILLs

In [4]:
path = "ILLs_official"
newILLs = ILLs;

#### wiki ILLs

In [10]:
path = "ILLs_wiki"
newILLs = wiki_ILLs;

### 生成文件

In [58]:
mt_newILLs = [getindex.(Ref(dict_words), pair) for pair in newILLs]
filter!(i->i[1]!=i[2], unique!(mt_newILLs))
println("对齐数目：\t", length.([newILLs, mt_newILLs])) # 13055 => 9802
dict_ILLs = ill2dict(mt_newILLs)

# 翻译三元组
mt_triples = triple_byMT.(triples)

# 对齐三元组
triple_byills(triple) = triple_byills(triple...)
triple_byills(sub, rel, obj) = (get(dict_ILLs, sub, sub), rel, get(dict_ILLs, obj, obj))
ill_triples = unique!(triple_byills.(mt_triples))
println("元组数目：\t", length.([triples, ill_triples]))

# 验证集
ill_valid_ners = [get(dict_ILLs, ner, ner) for ner in last.(mt_valid_ques_ner)]
ill_valid_ques = first.(mt_valid_ques_ner)

# 训练集
ill_train_ners = [get(dict_ILLs, ner, ner) for ner in getindex.(mt_train_ques_rels, 2)]
ill_train_ques = mt_train_ques;
ill_train_sols = [triple_byills.(triple_byMT.(getindex.(sols, Ref(2:4)))) for sols in train_sols]
ill_train_rels = [getindex.(sols, 2) for sols in ill_train_sols];

对齐数目：	[48131, 37119]
元组数目：	[267399, 263036]


### 测试

In [59]:
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in ill_triples
    push!(edges[sub], (rel, obj))
end

subs = unique!(first.(ill_triples))
# 第一跳
jumps = Dict(sub => [length(edges[sub])] for sub in subs)
# 第二跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][1] for (rel, obj) in edges[sub]))
end
# 第三跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][2] for (rel, obj) in edges[sub]))
end

In [60]:
# ILLs 3
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 37283, 48256]


(61, 137)

In [45]:
# 官方 ILLs
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 52268, 67437]


(156, 352)

In [25]:
# ILLs 1
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 44396, 57285]


(80, 209)

In [10]:
# ILLs 2
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 47455, 61439]


(98, 263)

In [8]:
# wiki ILLs
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

[0, 38070, 49921]


(6, 103)

In [7]:
function sort_count(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

sort_count (generic function with 1 method)

### 保存

In [61]:
open("EA_data/$path/triples.txt", "w") do io
    for triple in ill_triples
        println(io, join(triple, '\t'))
    end
end

open("EA_data/$path/train_ques.txt", "w") do io
    for (que, ner, rels) in zip(ill_train_ques, ill_train_ners, ill_train_rels)
        println(io, que, '\t', ner, '\t', join(rels, '\t'))
    end
end

open("EA_data/$path/train_data.txt", "w") do io
    for (que, ner, sols) in zip(ill_train_ques, ill_train_ners, ill_train_sols)
        println(io, que, '\t', ner)
        for sol in sols
            println(io, join(sol, '\t'))
        end
        println(io)
    end
end

open("EA_data/$path/valid_ques.txt", "w") do io
    for (que, ner) in zip(ill_valid_ques, ill_valid_ners)
        println(io, que, '\t', ner)
    end
end